In [1]:
"""

"""

In [ ]:
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages
import sys 
sys.path.append("../tools/")
from preprocess import * 
import pandas as pd

In [2]:
# Read CSV file with filenames, genres, etc.
info = pd.read_csv("train_info.csv")
imagelist, imagelist_wo = ListImages('\train_0')
train_info = info.loc[info['filename'].isin(imagelist_wo)]

In [ ]:
#save X without augmentation 
X = CreateDesignVector(imagelist)
#unflag this to store images to numpy 
#np.save('X_256.dat', X)

In [ ]:
#save Y without augmentation 
sorterIndex = dict(zip(imagelist_wo,range(len(imagelist_wo))))
train_info['filename_rank'] = train_info['filename'].map(sorterIndex)
train_info = train_info.sort_values(['filename_rank'], ascending = [True])
Y = train_info['genre'].values
#np.save('Y_genre.dat', X)

In [ ]:
Y = Y.astype(str)
Y, num_classes, dictY = label_class_to_Integer(Y)

In [ ]:
#split data 
percentage = 0.8
ntrain = len(X)
X_train = X[:ntrain]
Y_train = Y[:ntrain]
X_valid = X[ntrain:]
Y_valid = Y[ntrain:]

In [ ]:
Y

In [ ]:
#We have X and Y, now we do augmentation because the dataset is imbalanced
#For now we simply copy the data without any transformation 
#lets try 1000 first 
#ONLY Augment training data not the validation/test! 
XX = getFrequency_and_augment(X_train, Y_train, num_classes, 1000)
#now need to shuffle the data 
np.random.shuffle(XX)


#save training data with augmentation
np.save('X_256_augment_train', XX[:, :-1])
np.save('Y_genre_augment_train', XX[:, -1])

np.save('X_256_validation', X_valid)
np.save('Y_genre_validation', Y_valid)